# Twisted WS2 Bilayers

In [1]:
import sys
sys.path.append('../interfacemaster/')

In [2]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd

In [3]:
from pymatgen.core.structure import Structure

In [4]:
#sigmas, rotation angles, CNID areas, num of atoms
sigmas, thetas, A_cnid, anum = get_csl_twisted_graphenes(lim = 10, filename = \
                                                         'cif_files/WS2_mp-224_conventional_standard.cif',\
                                                        maxsigma = 2500, verbose=False)

/home/vscode/.local/lib/python3.10/site-packages/pymatgen/io/cif.py:1169: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [5]:
df = pd.DataFrame(column_stack((sigmas, thetas/pi*180, A_cnid, anum)), index = arange(len(sigmas))+1, \
                  columns = ['sigma','thetas','CNID area','atom number'])
df['sigma'] = df['sigma'].astype('int')
df['atom number'] = df['atom number'].astype('int')
df

sigma     thetas  CNID area  atom number
1       7  21.786789   1.259542           28
2      13  32.204228   0.678215           52
3      19  13.173551   0.464042           76
4      31  17.896551   0.284413          124
5      37   9.430008   0.238292          148
6      43  15.178179   0.205042          172
7      49  16.426421   0.179935          196
8      61   7.340993   0.144538          244
9      67  35.567302   0.131594          268
10     73  11.635051   0.120778          292
11     79  26.007824   0.111605          316
12     91   6.008983   0.096888          364
13     97  29.409311   0.090895          388
14    103  19.652860   0.085600          412
15    109  10.992733   0.080888          436
16    127   5.085848   0.069424          508
17    133   8.613238   0.066292          532
18    139  34.538944   0.063430          556
19    151  18.733998   0.058389          604
20    163  36.516938   0.054091          652
21    169   4.408455   0.052170          676
22    181  30.158276   0.048712          724
23    193   8.255621   0.045683          772
24    217   3.890238   0.040630          868
25    223  19.274807   0.039537          892
26    229  26.745652   0.038501          916
27    247  14.622221   0.035696          988
28    259  28.783203   0.034042         1036
29    271   3.481006   0.032534         1084
30    277  17.278244   0.031830         1108
31    283  24.016602   0.031155         1132
32    301   6.608610   0.029292         1204
33    331   3.149657   0.026637         1324
34    337  15.654144   0.026163         1348
35    367  11.985100   0.024024         1468
36    379  23.713582   0.023263         1516
37    403  14.307676   0.021878         1612
38    409  19.932013   0.021557         1636
39    457  27.052418   0.019293         1828
40    481  18.365765   0.018330         1924
41    523  20.146584   0.016858         2092
42    559  17.026049   0.015772         2236
43    613  23.301803   0.014383         2452

In [6]:
my_interface = core('cif_files/WS2_single_layer.cif',\
                    'cif_files/WS2_single_layer.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


/home/vscode/.local/lib/python3.10/site-packages/pymatgen/io/cif.py:1169: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


## Searching two_dimensional CSL

In [7]:
#a known target disorientation giving CSL
R = rot(a=[0,0,1],Theta = 18/180 * pi)

In [8]:
my_interface.parse_limit(du = 1e-3, S = 1e-3, sgm1=100, sgm2=100, dd = 1e-3)
#Do searching!
my_interface.search_one_position_2D(hkl_1=[0,0,1], hkl_2 = [0,0,1], theta_range = 2, dtheta= 0.01, pre_dt=True,pre_R=R, exact=False)

Congrates, we found an appx CSL!

U1 = 
[[-5  6]
 [ 1  5]
 [ 0  0]]; sigma_1 = 31

U2 = 
[[-6  5]
 [-1  6]
 [ 0  0]]; sigma_2 = 31

D = 
[[ 9.9999969e-01  7.5841000e-04 -0.0000000e+00]
 [-7.5825000e-04  9.9999974e-01  0.0000000e+00]
 [ 0.0000000e+00 -0.0000000e+00  1.0000000e+00]]

axis = [ 5.39873138e-16  9.35087575e-16 -8.81679743e+00] ; theta = 0.06



## Get bicrystal

In [9]:
hkl = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
my_interface.compute_bicrystal_two_D(hkl_1=[0,0,1], hkl_2 = [0,0,1],normal_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[ 0 -5 -6]
 [ 0  1 -5]
 [ 1  0  0]]
cell 2:
[[ 0 -6 -5]
 [ 0 -1 -6]
 [ 1  0  0]]


In [10]:
#layer_displacement = 3.4
#my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], \
                           #dp1 = 9.99905800e+00 - layer_displacement, filetype= 'VASP')
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], filetype= 'VASP', dp1 = 0)

## Sample CNID 

In [11]:
gridsize = 0.3
n1 = int(ceil(norm(my_interface.CNID[:,0])/gridsize))
n2 = int(ceil(norm(my_interface.CNID[:,1])/gridsize))

In [12]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
my_interface.sample_CNID([n1,n2], dp1 = 0)

CNID
[[-0.16129032  0.19354839]
 [ 0.03225806  0.16129032]
 [-0.         -0.        ]]
making 4 files...
completed


Please check the POSCAR file